In [2]:
!pip install praw
!pip install --upgrade praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 1.9 MB/s eta 0:00:00


In [11]:
import praw
import pandas as pd
import os
import time

# Initialize Reddit instance
reddit = praw.Reddit(client_id = "bBVDteGm6HjBNsTSTG_6bQ", #peronal use script
                    client_secret = "6KGSNj7r3TkjUCTZTX7nb7xfa5n0hA", #secret token
                    usernme = "KaushalSen", #profile username
                    password = "Fall@2023", #profile password
                    user_agent = "my-user-agent", #profile agent
                    check_for_async=False)

# URLs of Reddit posts to scrape
reddit_post_urls = [
    # Mohammed Imaduddin Ahmed
    'https://www.reddit.com/r/sysadmin/comments/199siwz/closet_chatgpt_user_at_work/',
    'https://www.reddit.com/r/ChatGPT/comments/133mc2v/what_do_you_all_actually_use_chatgpt_for/',
    'https://www.reddit.com/r/Professors/comments/18nmf24/typical_words_utilized_by_chatgpt/',
    'https://www.reddit.com/r/shortcuts/comments/135xfp6/i_got_my_chatgpt_shortcut_working_with_an_all/',
    'https://www.reddit.com/r/PhD/comments/127f9wv/how_do_you_use_chatgpt_when_writing_your_papers/',
    'https://www.reddit.com/r/technology/comments/161tyuj/chatgpt_generates_cancer_treatment_plans_that_are/',
    'https://www.reddit.com/r/education/comments/108i81c/should_we_stop_worrying_about_chatgpt_and_use_it/',
    # Ratna Srikar Duriseti
    'https://www.reddit.com/r/Damnthatsinteresting/comments/10sja83/3d_printer_does_homework_chatgpt_wrote/',
    'https://www.reddit.com/r/learnprogramming/comments/16wn6q9/chat_gpt_is_overrated/',
    'https://www.reddit.com/r/WGU/comments/157vonk/what_role_does_chatgpt_play_in_your_studying/',
    'https://www.reddit.com/r/ChatGPTCoding/comments/1b1eo9c/whats_the_coolest_coding_project_youve_built_with/',
    'https://www.reddit.com/r/Teachers/comments/zhkzp5/chatgpt_and_how_it_will_change_education/',
    'https://www.reddit.com/r/technology/comments/1113fyh/chatgpt_is_a_plague_upon_education/',
    'https://www.reddit.com/r/edtech/comments/1aqmy6k/is_chatgpt_banned_at_work_but_would_actually_be/',
    'https://www.reddit.com/r/Futurology/comments/12nx2mq/chatgpt_is_going_to_change_education_not_destroy/',
    'https://www.reddit.com/r/ChatGPT/comments/10a11w5/just_heard_a_chatgpt_school_speech/',
    'https://www.reddit.com/user/call_of_war_player/comments/16uc8la/what_would_the_world_look_like_without_us/',
    # Madhan Dadi
    'https://www.reddit.com/r/ChatGPT/comments/11jk3dk/turnitin_will_use_fingerprints_or_make_a_model_on/',
    'https://www.reddit.com/r/programming/comments/179upqc/report_chatgpts_impact_on_programming_education/',
    'https://www.reddit.com/r/ChatGPT/comments/13tx09i/only_2_of_us_adults_find_chatgpt_extremely_useful/',
    'https://www.reddit.com/r/ChatGPT/comments/13thzmm/high_school_english_teacher_looking_for/',
    # Sivanarayana Reddy
    'https://www.reddit.com/r/ChatGPT/comments/13r3v36/i_run_an_ai_tools_directory_here_are_some_top_ai/',
    'https://www.reddit.com/r/ChatGPT/comments/ztrpnc/im_a_teacher_and_i_love_chatgpt/',
    'https://www.reddit.com/r/Teachers/comments/130jd7t/why_i_boycotted_chatgpt/',
    'https://www.reddit.com/r/ChatGPT/comments/1178ff4/i_think_chatgpt_can_make_the_teaching_profession/',
    'https://www.reddit.com/r/UniUK/comments/13as7js/guys_stop_using_chatgpt_to_write_your_essays/',
    'https://www.reddit.com/r/ceo/comments/1arem6b/have_you_banned_chatgpt_at_work/',
    # Tanvi Kandula
    'https://www.reddit.com/r/IAmA/comments/11dg7ds/im_dr_wesley_wildman_a_professor_at_boston/',
    'https://www.reddit.com/r/ChatGPT/comments/1681xsu/theres_no_way_for_teachers_to_figure_out_if/',
    'https://www.reddit.com/r/Teachers/comments/zh43ju/respectfully_some_teachers_in_the_sub_are/',
    'https://www.reddit.com/r/pharmacy/comments/11vwtbp/chatgpt_will_replace_us_thoughts/',
    'https://www.reddit.com/r/singularity/comments/13euhi5/the_whole_idea_of_university_education_now_seems/',
    'https://www.reddit.com/r/ChatGPT/comments/11lsztp/hi_im_a_school_teacher_and_new_to_chatgpt_any/',
    'https://www.reddit.com/r/Genealogy/comments/11sgete/why_you_shouldnt_use_chatgpt_for_genealogical/',
    # Sandeep Reddy
    'https://www.reddit.com/r/Futurology/comments/10p1lgr/this_shouldnt_be_a_surprise_the_education/',
    'https://www.reddit.com/r/ChatGPT/comments/12q2b0e/chatgpt_helped_me_pass_an_exam_with_94_despite/',
    'https://www.reddit.com/r/EngineeringStudents/comments/11ujrmh/how_do_you_use_chatgpt_as_an_engineer_student/',
    'https://www.reddit.com/r/ChatGPT/comments/12rt45l/im_a_high_school_math_teacher_and_just_showed_all/',
    'https://www.reddit.com/r/ChatGPT/comments/11z2som/ive_been_letting_my_high_school_students_use/',
    'https://www.reddit.com/r/Python/comments/15fo9wl/i_am_teaching_kids_how_to_code_and_the_kids_are/'
]


structured_comments = []

def process_comment_tree(comment, submission, depth=0, parent_id=None):
    """
    Recursively process a comment and its replies, appending each to the
    structured_comments list with depth and parent_id to maintain structure.
    """
    structured_comments.append({
        'post_id': submission.id,
        'post_title': submission.title,
        'post_score': submission.score,
        'post_url': submission.url,
        'comment_id': comment.id,
        'parent_id': parent_id,
        'comment_depth': depth,
        'comment_body': comment.body,
        'comment_likes': comment.score
    })
    if hasattr(comment, 'replies'):
        for reply in comment.replies:
            process_comment_tree(reply, submission, depth + 1, comment.id)

def extract_data_from_url(url):
    submission_id = url.split("/")[6]
    submission = reddit.submission(id=submission_id)
    submission.comments.replace_more(limit=None)  # limit=None for full depth, might take longer
    for top_level_comment in submission.comments:
        process_comment_tree(top_level_comment,submission)

for url in reddit_post_urls:
    extract_data_from_url(url)
    time.sleep(5)

# Convert the structured comments to a DataFrame
df = pd.DataFrame(structured_comments)

# Ensure the script's directory has a subdirectory named 'data'
os.makedirs('data', exist_ok=True)

# Save DataFrame to CSV
csv_file_path = os.path.join('data', 'reddit_comments_structured_final_reviews.csv')
df.to_csv(csv_file_path, index=False)

print(f"Data saved to {csv_file_path}")
print(len(structured_comments))


Data saved to data/reddit_comments_structured_final.csv
17130
